In [ ]:
%%time
import pyodbc 
import pandas as pd
import numpy as np
import datetime
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
import timeit

warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_columns', None)
#!pip install jupyter_contrib_nbextensions




In [ ]:
%%time
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=LAPTOP-CU20T8L1;"
                      "Database=CAMPAIGN_DB2;"
                      "Trusted_Connection=yes;")


cursor = cnxn.cursor()
sql1 = pd.read_sql_query('SELECT TOP(1000) * FROM CAMPAIGN_TBL_FINAL',cnxn)
df1 = pd.DataFrame(sql1)

In [ ]:
%%time
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=LAPTOP-CU20T8L1;"
                      "Database=CAMPAIGN_DB;"
                      "Trusted_Connection=yes;")


cursor = cnxn.cursor()
sql2 = pd.read_sql_query('SELECT TOP(1000) * FROM CAMPAIGN_TBL_FINAL',cnxn)
df2 = pd.DataFrame(sql2)

In [ ]:
%%time
df = pd.concat([df1,df2])
df.drop_duplicates() #dropping duplicates if any

In [ ]:
df.shape

In [9]:
%%time
df['Bet Time'] = pd.to_datetime(df['Bet Time'])
df['Result Time'] = pd.to_datetime(df['Result Time'])
df['Registered Date'] = pd.to_datetime(df['Registered Date'])
df['Last Login'] = pd.to_datetime(df['Last Login'])

df['Transaction Date'] = df['Bet Time'].dt.date
df['Transaction Month'] = pd.to_datetime(df['Bet Time']).dt.to_period('M')

df['days_since_last_login'] = (df['Last Login'].max() - df['Last Login']).astype('timedelta64[D]')
df['Age_on_platform'] = (df['Last Login']- df['Registered Date']).astype('timedelta64[D]')
 
last_betting_date = df.groupby(['Player ID'])['Bet Time'].agg(['max']).reset_index().rename(columns={"max":"Last Betting Date"})
avg_no_of_bets_per_day = df.groupby(['Player ID','Transaction Date'])['Player ID'].count().groupby('Player ID').agg(['mean']).reset_index().rename(columns={"mean":"Avg Bets per Day"}) #sum/count
avg_no_of_bets_per_month = df.groupby(['Player ID','Transaction Month'])['Player ID'].count().groupby('Player ID').agg(['mean']).reset_index().rename(columns={"mean":"Avg Bets per Month"}) #sum/count

avg_amount_on_bets_per_day = df.groupby(['Player ID','Transaction Date'])['Stake'].sum().groupby('Player ID').agg(['mean']).reset_index().rename(columns={"mean":"Avg Bets amount per Day"})
avg_amount_on_bets_per_month = df.groupby(['Player ID','Transaction Month'])['Stake'].sum().groupby('Player ID').agg(['mean']).reset_index().rename(columns={"mean":"Avg Bets amount per Month"})

no_of_visiting_days = df.groupby(['Player ID', 'Transaction Date']).size().groupby('Player ID').count().reset_index().rename(columns={0:'No_of_visit_days'})

time_df = pd.DataFrame()
temp_df = df[['Player ID','days_since_last_login','Age_on_platform','Bet Time', 'Result Time','Registered Date',
       'Last Login']].groupby('Player ID').agg(['max']).reset_index() 
temp_df.columns = ['Player ID','days_since_last_login','Age_on_platform','Bet Time', 'Result Time','Registered Date','Last Login']

dataframes = [last_betting_date,avg_no_of_bets_per_day,avg_amount_on_bets_per_day,avg_no_of_bets_per_month,avg_amount_on_bets_per_month,no_of_visiting_days,temp_df]
time_df = reduce(lambda left, right: pd.merge(left, right, on='Player ID', how='left'), dataframes)
time_df['loyalty_ratio'] = time_df['No_of_visit_days']/time_df['Age_on_platform']



bet_df = df.groupby('Player ID').agg( 
                                        total_stake=pd.NamedAgg(column='Stake', aggfunc='sum'),
                                        mean_stake=pd.NamedAgg(column='Stake', aggfunc='mean'),
                                        min_stake = pd.NamedAgg(column='Stake', aggfunc='min'),
                                        max_stake = pd.NamedAgg(column='Stake', aggfunc='max'),
                                        bets = pd.NamedAgg(column='Stake', aggfunc='count'),
                                        total_Valid_Bet =pd.NamedAgg(column='Valid Bet', aggfunc='sum'),
                                        mean_Valid_Bet=pd.NamedAgg(column='Valid Bet', aggfunc='mean'),    
                                        total_WIn_Loss =pd.NamedAgg(column='Win/Loss', aggfunc='sum'),
                                        mean_total_Win_Loss=pd.NamedAgg(column='Win/Loss', aggfunc='mean')).reset_index()


bet_df.columns = ['Player ID','Total Stake','Avg Stake(Total Stake/Total Bets)','Min Stake','Max Stake','Total Bets','Total Valid Bet','Avg Valid Bet','Total Win/Loss', 'Avg Win/Loss']


# game_group = df.groupby(['Player ID', 'Game Group']).size().reset_index() #['Game Group'].agg('count')
# game_group = game_group.rename(columns ={0:'Counts'})

game_group_pref = df[['Player ID', 'Game', 'Game Group','Provider','Product Type']]
temp_col1= [x for x in game_group_pref['Game Group'].unique()]
for x in temp_col1:
    game_group_pref[x]= np.where(game_group_pref['Game Group'] == x,1,0)
game_group_df = game_group_pref.groupby('Player ID').agg(['sum']).reset_index()[['Player ID']+temp_col1]
game_group_df.columns = ['Player ID']+temp_col1

game_provider_type_pref = df[['Player ID', 'Game', 'Game Group','Provider','Product Type']]
temp_col1= [x for x in game_provider_type_pref['Provider'].unique()]
for x in temp_col1:
    game_provider_type_pref[x]= np.where(game_provider_type_pref['Provider'] == x,1,0)
    
game_provider_type_pref_df = game_provider_type_pref.groupby('Player ID').agg(['sum']).reset_index()[['Player ID']+temp_col1]
game_provider_type_pref_df.columns = ['Player ID']+temp_col1

game_product_type_pref = df[['Player ID', 'Game', 'Game Group','Provider','Product Type']]
temp_col1= [x for x in game_product_type_pref['Product Type'].unique()]
for x in temp_col1:
    game_product_type_pref[x]= np.where(game_product_type_pref['Product Type'] == x,1,0)
    
game_product_type_pref_df = game_product_type_pref.groupby('Player ID').agg(['sum']).reset_index()[['Player ID']+temp_col1]
game_product_type_pref_df.columns = ['Player ID']+temp_col1




game_pref_1 = df.groupby('Player ID').agg(
    FavFirst=pd.NamedAgg(column='Game',aggfunc= lambda x: x.value_counts().index[0] if len(x.value_counts()) else None),
    FavFirstFrequency=pd.NamedAgg(column='Game',aggfunc= lambda x: x.value_counts()[0] if len(x.value_counts()) else None)                                         
).reset_index()


game_pref_2 = df.groupby('Player ID').agg(
    FavSecond=pd.NamedAgg(column='Game',aggfunc= lambda x: x.value_counts().index[1] if len(x.value_counts())>1 else None),
    FavSecondFrequency=pd.NamedAgg(column='Game',aggfunc= lambda x: x.value_counts()[1] if len(x.value_counts())>1 else None)                                         
).reset_index()



game_pref_3 = df.groupby('Player ID').agg(
    FavThird=pd.NamedAgg(column='Game',aggfunc= lambda x: x.value_counts().index[2] if len(x.value_counts())>2 else None),
    FavThirdFrequency=pd.NamedAgg(column='Game',aggfunc= lambda x: x.value_counts()[2] if len(x.value_counts())>2 else None)                                         
).reset_index()


game_pref_4 = df.groupby('Player ID').agg(
    FavFourth=pd.NamedAgg(column='Game',aggfunc= lambda x: x.value_counts().index[3] if len(x.value_counts())>3 else None),
    FavFourthFrequency=pd.NamedAgg(column='Game',aggfunc= lambda x: x.value_counts()[3] if len(x.value_counts())>3 else None)                                         
).reset_index()


game_pref_5 = df.groupby('Player ID').agg(
    FavFifth=pd.NamedAgg(column='Game',aggfunc= lambda x: x.value_counts().index[4] if len(x.value_counts())>4 else None),
    FavFifthFrequency=pd.NamedAgg(column='Game',aggfunc= lambda x: x.value_counts()[4] if len(x.value_counts())>4 else None)                                         
).reset_index()



dataframes = [game_pref_1,game_pref_2,game_pref_3,game_pref_4,game_pref_5]
game_pref_df = reduce(lambda left, right: pd.merge(left, right, on='Player ID', how='left'), dataframes)

game_pref_df['FavFirstAmount'] =0.0
game_pref_df['FavSecondAmount'] =0.0
game_pref_df['FavThirdAmount'] =0.0
game_pref_df['FavFourthAmount'] =0.0
game_pref_df['FavFifthAmount'] =0.0

temp_game_df = df.groupby(['Player ID','Game'])['Stake'].sum().reset_index()

for player in game_pref_df["Player ID"]:
    #print(player + "    "+ "---->")
    for game in  temp_game_df[temp_game_df['Player ID']==player]['Game']:
       # print(game)
        if  game_pref_df[game_pref_df['Player ID']==player]['FavFirst'].values == game:
             
            game_pref_df.at[game_pref_df['Player ID']==player, 'FavFirstAmount'] = temp_game_df[temp_game_df['Player ID']==player][temp_game_df[temp_game_df['Player ID']==player]['Game']==game]['Stake'].values
           # print('pusheddata')
            
        if  game_pref_df[game_pref_df['Player ID']==player]['FavSecond'].values == game:
             
            game_pref_df.at[game_pref_df['Player ID']==player, 'FavSecondAmount'] = temp_game_df[temp_game_df['Player ID']==player][temp_game_df[temp_game_df['Player ID']==player]['Game']==game]['Stake'].values
            #print('pusheddata')
            
        if  game_pref_df[game_pref_df['Player ID']==player]['FavThird'].values == game:
             
            game_pref_df.at[game_pref_df['Player ID']==player, 'FavThirdAmount'] = temp_game_df[temp_game_df['Player ID']==player][temp_game_df[temp_game_df['Player ID']==player]['Game']==game]['Stake'].values
            #print('pusheddata')
            
        if  game_pref_df[game_pref_df['Player ID']==player]['FavFourth'].values == game:
             
            game_pref_df.at[game_pref_df['Player ID']==player, 'FavFourthAmount'] = temp_game_df[temp_game_df['Player ID']==player][temp_game_df[temp_game_df['Player ID']==player]['Game']==game]['Stake'].values
            #print('pusheddata')
            
        if  game_pref_df[game_pref_df['Player ID']==player]['FavFifth'].values == game:
             
            game_pref_df.at[game_pref_df['Player ID']==player, 'FavFifthAmount'] = temp_game_df[temp_game_df['Player ID']==player][temp_game_df[temp_game_df['Player ID']==player]['Game']==game]['Stake'].values
            #print('pusheddata')
        

dataframes = [game_product_type_pref_df,game_provider_type_pref_df,game_group_df,game_pref_df]
game_df = reduce(lambda left, right: pd.merge(left, right, on='Player ID', how='left'), dataframes)

dataframes = [time_df,bet_df,game_df]
final_df = reduce(lambda left, right: pd.merge(left, right, on='Player ID', how='left'), dataframes)


Wall time: 928 ms


In [10]:
final_df.columns

Index(['Player ID', 'Last Betting Date', 'Avg Bets per Day',
       'Avg Bets amount per Day', 'Avg Bets per Month',
       'Avg Bets amount per Month', 'No_of_visit_days',
       'days_since_last_login', 'Age_on_platform', 'Bet Time', 'Result Time',
       'Registered Date', 'Last Login', 'loyalty_ratio', 'Total Stake',
       'Avg Stake(Total Stake/Total Bets)', 'Min Stake', 'Max Stake',
       'Total Bets', 'Total Valid Bet', 'Avg Valid Bet', 'Total Win/Loss',
       'Avg Win/Loss', 'EGAME', 'SPORTS', 'LIVE', 'MPG_x', 'MG', 'VT',
       'SBOBET', 'AMEBA', 'MX', 'RT', 'CQ9', 'GGAMING', 'VENUS', 'SA', 'SABA',
       'PT', 'SLOTGAME', 'SPORTSBOOK', 'BACCARAT', 'MPG_y', 'DRAGON_TIGER',
       'FISHING', 'FavFirst', 'FavFirstFrequency', 'FavSecond',
       'FavSecondFrequency', 'FavThird', 'FavThirdFrequency', 'FavFourth',
       'FavFourthFrequency', 'FavFifth', 'FavFifthFrequency', 'FavFirstAmount',
       'FavSecondAmount', 'FavThirdAmount', 'FavFourthAmount',
       'FavFifthAmount']